In [3]:
import tkinter as tk
from tkinter import filedialog, Label
from PIL import Image, ImageTk
import numpy as np
import cv2
import tensorflow as tf
import time
from datetime import datetime

model = tf.keras.models.load_model("asl_model.h5")

model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

class_names = list('ABCDEFGHIJKLMNOPQRSTUVWXYZ') + ['del', 'nothing', 'space']

def preprocess_image(image_path):
    img = Image.open(image_path).resize((64, 64))
    img_array = np.array(img) / 255.0
    return np.expand_dims(img_array, axis=0)

def preprocess_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.resize(frame, (64, 64))
        frame = frame / 255.0
        frames.append(frame)
    cap.release()
    return np.array(frames)

def is_within_time():
    current_time = datetime.now().time()
    start_time = datetime.strptime("18:00", "%H:%M").time()
    end_time = datetime.strptime("22:00", "%H:%M").time()
    return start_time <= current_time <= end_time

def load_image():
    if not is_within_time():
        result_label.config(text="Predictions are only allowed between 6 PM and 10 PM")
        return

    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    
    img = Image.open(file_path)
    img.thumbnail((300, 300))
    img = ImageTk.PhotoImage(img)
    panel.config(image=img)
    panel.image = img

    img_array = preprocess_image(file_path)
    prediction = model.predict(img_array)
    predicted_class = class_names[np.argmax(prediction)]
    
    result_label.config(text=f'Predicted Sign: {predicted_class}')

def load_video():
    if not is_within_time():
        result_label.config(text="Predictions are only allowed between 6 PM and 10 PM")
        return

    file_path = filedialog.askopenfilename()
    if not file_path:
        return
    
    cap = cv2.VideoCapture(file_path)
    ret, frame = cap.read()
    cap.release()
    if ret:
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        frame = Image.fromarray(frame)
        frame.thumbnail((300, 300))
        frame = ImageTk.PhotoImage(frame)
        panel.config(image=frame)
        panel.image = frame

    video_frames = preprocess_video(file_path)
    predictions = model.predict(video_frames)
    predicted_classes = [class_names[np.argmax(pred)] for pred in predictions]

    predicted_sentence = " ".join(predicted_classes)
    
    result_label.config(text=f'Predicted Sentence: {predicted_sentence}')

root = tk.Tk()
root.title("Task 2 - Sign Language Predictor")
root.geometry("600x600")

title_label = Label(root, text="Task 2", font=("Helvetica", 24, "bold"))
title_label.pack(pady=10)
subtitle_label = Label(root, text="Sign Language Predictor", font=("Helvetica", 18))
subtitle_label.pack(pady=5)

btn_image = tk.Button(root, text="Load Image", command=load_image)
btn_image.pack(pady=10)

btn_video = tk.Button(root, text="Load Video", command=load_video)
btn_video.pack(pady=10)

panel = Label(root)
panel.pack(pady=10)

result_label = Label(root, text="Predicted Sign: ", font=("Helvetica", 16))
result_label.pack(pady=20)

root.mainloop()
